In [1]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt
# import itertools

%matplotlib inline

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
# from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
# from keras.callbacks import ReduceLROnPlateau
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense , Activation

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import matthews_corrcoef


import seaborn as sns

import time

from xgboost import XGBClassifier

import lightgbm as lgb

In [2]:
products = {}
count = 1
for i in range(92):
    df = pd.read_csv('../input/products/product_{}.csv'.format(str(count)))
    products['product_{}'.format(str(count))] = df
    print('Done!{}'.format(str(count)))
    count+=1

Done!1
Done!2
Done!3
Done!4
Done!5
Done!6
Done!7
Done!8
Done!9
Done!10
Done!11
Done!12
Done!13
Done!14
Done!15
Done!16
Done!17
Done!18
Done!19
Done!20
Done!21
Done!22
Done!23
Done!24
Done!25
Done!26
Done!27
Done!28
Done!29
Done!30
Done!31
Done!32
Done!33
Done!34
Done!35
Done!36
Done!37
Done!38
Done!39
Done!40
Done!41
Done!42
Done!43
Done!44
Done!45
Done!46
Done!47
Done!48
Done!49
Done!50
Done!51
Done!52
Done!53
Done!54
Done!55
Done!56
Done!57
Done!58
Done!59
Done!60
Done!61
Done!62
Done!63
Done!64
Done!65
Done!66
Done!67
Done!68
Done!69
Done!70
Done!71
Done!72
Done!73
Done!74
Done!75
Done!76
Done!77
Done!78
Done!79
Done!80
Done!81
Done!82
Done!83
Done!84
Done!85
Done!86
Done!87
Done!88
Done!89
Done!90
Done!91
Done!92


In [3]:
products['product_1'].head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S29_F3466_T1,L3_S29_F3469_T1,L3_S29_F3472_T1,L3_S29_F3475_T1,L3_S29_F3478_T1,L3_S29_F3481_T1,L3_S29_F3484_T1,L3_S29_F3487_T1,L3_S29_F3490_T1,L3_S29_F3493_T1
0,365,-0.010,0.019,-0.015,0.003,0.031,0.116,0.000,-0.032,-0.031,...,1,1,1,1,1,1,1,1,1,1
1,409,-0.068,-0.108,0.003,-0.016,0.118,0.025,0.015,0.128,0.046,...,0,0,0,0,0,0,0,0,0,0
2,502,-0.049,-0.078,0.003,-0.016,0.074,0.161,0.015,0.088,-0.056,...,1,1,1,1,1,1,1,1,1,1
3,584,0.010,0.063,-0.015,-0.034,0.031,0.070,0.000,-0.072,0.173,...,0,0,0,0,0,0,0,0,0,0
4,1036,0.095,0.071,-0.179,-0.179,0.031,0.161,-0.030,-0.112,0.071,...,1,1,1,1,1,1,1,1,1,1


In [4]:
def drop_id(df):
    df.drop('Id' , 1 , inplace = True)

In [5]:
scaler = StandardScaler()
def scaling(X):
    # scaling the data
    scaled_features = scaler.fit_transform(X)
    X = pd.DataFrame(scaled_features, index=X.index, columns=X.columns)
    return X

In [6]:
smt = SMOTE(random_state=42)
def sampling(X, y):
    X_res, y_res = smt.fit_resample(X, y)
    return X_res , y_res

In [7]:
def process_data(df):
    drop_id(df)
    X = df.drop('Response' , 1)
    y = df['Response']
    X_res , y_res = sampling(X, y)
    X_train , X_test , y_train , y_test = train_test_split(X_res , y_res, test_size = 0.2 , random_state = 42)
    X_train = scaling(X_train)
    X_test = scaling(X_test)
    return X_train, X_test, y_train, y_test, X

In [8]:
time_dict = {}
train_acc_dict = {}
test_acc_dict = {}
matrix = {}
feature_imp_dict = {}

def storing(train_acc , test_acc , time_taken , mat, product_name, feature_imp,y_test ,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test ,y_pred).ravel()
    time_dict[product_name] = time_taken
    train_acc_dict[product_name] = train_acc
    test_acc_dict[product_name] = test_acc
    feature_imp_dict[product_name] = feature_imp
    matrix[product_name] = [tn, fp, fn, tp]

In [9]:
def model_training(X_train, X_test, y_train , y_test, product_name):
    model = lgb.LGBMClassifier()
    start = time.time()
    model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)],
          verbose=20,eval_metric='logloss')
    end = time.time()
    time_taken = end - start
    train_acc = model.score(X_train,y_train)
    test_acc = model.score(X_test,y_test)
    y_pred = model.predict(X_test)
    mat = confusion_matrix(y_test, y_pred)
    feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_,X.columns)), columns=['Value','Feature'])
    storing(train_acc , test_acc , time_taken , mat, product_name, feature_imp,y_test ,y_pred)

In [10]:
for i in products.keys():
    product_name = i
    df = products[i]
    X_train, X_test, y_train, y_test, X = process_data(df)
    model_training(X_train, X_test, y_train , y_test, product_name)
    print('Done!{}'.format(product_name) , '\n')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[20]	training's binary_logloss: 0.0962363	valid_0's binary_logloss: 0.91169
[40]	training's binary_logloss: 0.0219729	valid_0's binary_logloss: 1.29115
[60]	training's binary_logloss: 0.00636066	valid_0's binary_logloss: 1.32271
[80]	training's binary_logloss: 0.00265392	valid_0's binary_logloss: 1.2513
[100]	training's binary_logloss: 0.00160508	valid_0's binary_logloss: 1.25618
Done!product_1 



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[20]	training's binary_logloss: 0.0979643	valid_0's binary_logloss: 1.13846
[40]	training's binary_logloss: 0.0210556	valid_0's binary_logloss: 1.32211
[60]	training's binary_logloss: 0.00631883	valid_0's binary_logloss: 1.24515
[80]	training's binary_logloss: 0.00290234	valid_0's binary_logloss: 0.976927
[100]	training's binary_logloss: 0.00197593	valid_0's binary_logloss: 0.76544
Done!product_2 



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

[20]	training's binary_logloss: 0.0914182	valid_0's binary_logloss: 1.08393
[40]	training's binary_logloss: 0.0219863	valid_0's binary_logloss: 1.44854
[60]	training's binary_logloss: 0.00750975	valid_0's binary_logloss: 2.11127
[80]	training's binary_logloss: 0.00398683	valid_0's binary_logloss: 2.22299
[100]	training's binary_logloss: 0.00301025	valid_0's binary_logloss: 2.09283
Done!product_90 



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[20]	training's binary_logloss: 0.103236	valid_0's binary_logloss: 0.988422
[40]	training's binary_logloss: 0.0277357	valid_0's binary_logloss: 1.21949
[60]	training's binary_logloss: 0.00913957	valid_0's binary_logloss: 1.50144
[80]	training's binary_logloss: 0.00433234	valid_0's binary_logloss: 1.70588
[100]	training's binary_logloss: 0.00286638	valid_0's binary_logloss: 1.81602
Done!product_91 



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[20]	training's binary_logloss: 0.096134	valid_0's binary_logloss: 1.3884
[40]	training's binary_logloss: 0.0253081	valid_0's binary_logloss: 1.99239
[60]	training's binary_logloss: 0.00845163	valid_0's binary_logloss: 1.79982
[80]	training's binary_logloss: 0.00387576	valid_0's binary_logloss: 1.78892
[100]	training's binary_logloss: 0.0024913	valid_0's binary_logloss: 1.77157
Done!product_92 



In [11]:
time_dict['product_1']

7.480327129364014

In [12]:
feature_imp_dict['product_1']

,Value,Feature
0,0,L0_S0_D11
1,0,L0_S0_D13
2,0,L0_S0_D15
3,0,L0_S0_D17
4,0,L0_S0_D19
...,...,...
419,71,L0_S0_F16
420,72,L0_S0_F8
421,79,L3_S29_F3357
422,82,L0_S6_F118


In [13]:
matrix

{'product_1': [877, 1711, 0, 2530],
 'product_2': [1016, 1487, 2, 2427],
 'product_3': [1701, 911, 0, 2496],
 'product_4': [292, 2305, 0, 2448],
 'product_5': [1511, 1084, 0, 2552],
 'product_6': [1566, 858, 0, 2451],
 'product_7': [682, 11, 0, 683],
 'product_8': [2071, 366, 0, 2431],
 'product_9': [836, 1663, 0, 2427],
 'product_10': [548, 3, 29, 474],
 'product_11': [45, 4, 0, 37],
 'product_12': [369, 3184, 0, 3487],
 'product_13': [499, 2001, 0, 2434],
 'product_14': [1247, 2386, 22, 3635],
 'product_15': [1066, 1384, 3, 2493],
 'product_16': [614, 1277, 4, 1886],
 'product_17': [80, 2492, 0, 2439],
 'product_18': [1120, 1433, 1, 2469],
 'product_19': [2015, 443, 1, 2484],
 'product_20': [503, 1, 4, 473],
 'product_21': [785, 1801, 2, 2502],
 'product_22': [711, 1838, 0, 2495],
 'product_23': [1412, 443, 1, 1842],
 'product_24': [925, 1549, 0, 2467],
 'product_25': [465, 2027, 0, 2481],
 'product_26': [719, 1770, 0, 2443],
 'product_27': [1878, 712, 0, 2511],
 'product_28': [1656,

In [14]:
test_acc_dict

{'product_1': 0.6656897225478703,
 'product_2': 0.6980940794809408,
 'product_3': 0.8216523101018011,
 'product_4': 0.5431119920713577,
 'product_5': 0.7893918787643287,
 'product_6': 0.824,
 'product_7': 0.9920058139534884,
 'product_8': 0.9248151191454396,
 'product_9': 0.6624035728786033,
 'product_10': 0.969639468690702,
 'product_11': 0.9534883720930233,
 'product_12': 0.5477272727272727,
 'product_13': 0.5944466963923795,
 'product_14': 0.669684499314129,
 'product_15': 0.7195713708046907,
 'product_16': 0.6612007405448294,
 'product_17': 0.5026940730393135,
 'product_18': 0.7145132391001393,
 'product_19': 0.9101760064738014,
 'product_20': 0.9949031600407747,
 'product_21': 0.6457760314341847,
 'product_22': 0.6356066613798572,
 'product_23': 0.8799351000540833,
 'product_24': 0.6865007083586319,
 'product_25': 0.592398954353509,
 'product_26': 0.6411192214111923,
 'product_27': 0.860419525583219,
 'product_28': 0.8474435934589112,
 'product_29': 0.9106012658227848,
 'product_3

In [15]:
sum = 0
for i in test_acc_dict.keys():
    test_acc = test_acc_dict[i]
    sum += test_acc
    
mean = sum/92
print(mean)

0.8115787106952681


In [16]:
products_to_optimize = []
for i in test_acc_dict.keys():
    test_acc = test_acc_dict[i]
    if test_acc < 0.85:
        products_to_optimize.append(i)

In [17]:
sum_tn = 0
sum_fp = 0
sum_fn = 0
sum_tp = 0

for i in matrix.keys():
    tn = matrix[i][0]
    fp = matrix[i][1]
    fn = matrix[i][2]
    tp = matrix[i][3]
    sum_tn += tn
    sum_fp += fp
    sum_fn += fn
    sum_tp += tp
    

In [18]:
print(sum_tn)
print(sum_fp)
print(sum_fn)
print(sum_tp)


107417
75308
276
179208


In [19]:
x = (179208 + 75308) * (179208 + 276) * (107417 + 75308) * (107417 + 276)
print(((179208 * 107417) - (75308 * 276)) / (x)**0.5)

0.6413544251344886


In [20]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [21]:
for i in products_to_optimize:
    df = products[i]
    X = df.drop('Response' , 1)
    y = df['Response']
    X_res , y_res = sampling(X, y)
    print(i , ':', '\n' , y_res.value_counts(), '\n')

product_1 : 
 0    12794
1    12794
Name: Response, dtype: int64 

product_2 : 
 0    12329
1    12329
Name: Response, dtype: int64 

product_3 : 
 0    12768
1    12768
Name: Response, dtype: int64 

product_4 : 
 0    12611
1    12611
Name: Response, dtype: int64 

product_5 : 
 0    12866
1    12866
Name: Response, dtype: int64 

product_6 : 
 0    12186
1    12186
Name: Response, dtype: int64 

product_9 : 
 0    12313
1    12313
Name: Response, dtype: int64 

product_12 : 
 0    17599
1    17599
Name: Response, dtype: int64 

product_13 : 
 0    12335
1    12335
Name: Response, dtype: int64 

product_14 : 
 0    18224
1    18224
Name: Response, dtype: int64 

product_15 : 
 0    12363
1    12363
Name: Response, dtype: int64 

product_16 : 
 0    9451
1    9451
Name: Response, dtype: int64 

product_17 : 
 0    12527
1    12527
Name: Response, dtype: int64 

product_18 : 
 0    12557
1    12557
Name: Response, dtype: int64 

product_21 : 
 0    12723
1    12723
Name: Response, dtyp

In [22]:
max_acc_dict = {}
min_acc_dict = {}
overall_acc_dict = {}
std_dev_dict = {}

def storing(max_acc,min_acc ,overall_acc , std_dev, product_name):
    max_acc_dict[product_name] = max_acc
    min_acc_dict[product_name] = min_acc
    overall_acc_dict[product_name] = overall_acc
    std_dev_dict[product_name] = std_dev

In [23]:
def stratified_model_training(X, y , product_name):
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
    acc_stratified = []
    model = lgb.LGBMClassifier()
    X_scaled = scaling(X)
    for train_index, test_index in skf.split(X, y):
        X_train_fold, X_test_fold = X_scaled.iloc[train_index], X_scaled.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train_fold, y_train_fold)
        acc_stratified.append(model.score(X_test_fold, y_test_fold))
        max_acc = max(acc_stratified)*100
        min_acc = min(acc_stratified)*100
        overall_acc = np.array(acc_stratified).mean()*100
        std_dev = np.std(acc_stratified)
        storing(max_acc,min_acc ,overall_acc , std_dev, product_name)
    print(product_name, 'Done!')

In [24]:
for i in products_to_optimize:
    product_name = i
    df = products[i]
    X = df.drop('Response' , 1)
    y = df['Response']
    X_res , y_res = sampling(X, y)
    stratified_model_training(X_res, y_res , product_name)

product_1 Done!
product_2 Done!
product_3 Done!
product_4 Done!
product_5 Done!
product_6 Done!
product_9 Done!
product_12 Done!
product_13 Done!
product_14 Done!
product_15 Done!
product_16 Done!
product_17 Done!
product_18 Done!
product_21 Done!
product_22 Done!
product_24 Done!
product_25 Done!
product_26 Done!
product_28 Done!
product_30 Done!
product_34 Done!
product_44 Done!
product_46 Done!
product_49 Done!
product_50 Done!
product_54 Done!
product_55 Done!
product_59 Done!
product_60 Done!
product_61 Done!
product_62 Done!
product_63 Done!
product_65 Done!
product_66 Done!
product_72 Done!
product_73 Done!
product_78 Done!
product_79 Done!
product_80 Done!
product_82 Done!
product_83 Done!
product_86 Done!
product_87 Done!
product_88 Done!
product_90 Done!
product_91 Done!
product_92 Done!


In [25]:
max_acc_dict

{'product_1': 99.88272087568413,
 'product_2': 99.87834549878345,
 'product_3': 99.84338292873923,
 'product_4': 99.84139571768438,
 'product_5': 99.88344988344988,
 'product_6': 99.75389663658737,
 'product_9': 99.8375964271214,
 'product_12': 99.6590909090909,
 'product_13': 99.87839481151197,
 'product_14': 99.6159122085048,
 'product_15': 99.87868985038415,
 'product_16': 100.0,
 'product_17': 99.80047885075818,
 'product_18': 99.92035045798487,
 'product_21': 99.88207547169812,
 'product_22': 99.80174464710547,
 'product_24': 99.87859166329422,
 'product_25': 99.8793242156074,
 'product_26': 99.87834549878345,
 'product_28': 99.95859213250517,
 'product_30': 99.8805256869773,
 'product_34': 99.78021978021978,
 'product_44': 99.83733224888167,
 'product_46': 99.89183342347215,
 'product_49': 99.88221436984688,
 'product_50': 99.91883116883116,
 'product_54': 99.8435054773083,
 'product_55': 99.89281886387997,
 'product_59': 99.8960498960499,
 'product_60': 99.94658119658119,
 'prod

In [26]:
min_acc_dict

{'product_1': 99.64816262705239,
 'product_2': 99.7161394971614,
 'product_3': 99.56930305403289,
 'product_4': 99.60364645263576,
 'product_5': 99.53361834434511,
 'product_6': 99.46655724251129,
 'product_9': 99.6751928542428,
 'product_12': 99.375,
 'product_13': 99.67571949736522,
 'product_14': 99.42386831275721,
 'product_15': 99.59546925566343,
 'product_16': 99.73544973544973,
 'product_17': 99.60079840319361,
 'product_18': 99.60175228992433,
 'product_21': 99.60691823899371,
 'product_22': 99.48453608247422,
 'product_24': 99.55465587044534,
 'product_25': 99.6379726468222,
 'product_26': 99.59432048681542,
 'product_28': 99.58609271523179,
 'product_30': 99.68140183193947,
 'product_34': 98.68131868131869,
 'product_44': 99.51219512195122,
 'product_46': 99.56709956709958,
 'product_49': 99.52904238618524,
 'product_50': 99.39123376623377,
 'product_54': 99.05956112852664,
 'product_55': 99.57127545551982,
 'product_59': 99.58419958419958,
 'product_60': 99.73290598290599,
 

In [27]:
overall_acc_dict

{'product_1': 99.75769952651439,
 'product_2': 99.80939314227243,
 'product_3': 99.72587564923542,
 'product_4': 99.73436611563781,
 'product_5': 99.7629347376724,
 'product_6': 99.6635459687488,
 'product_9': 99.77260366503043,
 'product_12': 99.4744027551732,
 'product_13': 99.78516416700444,
 'product_14': 99.55004223640643,
 'product_15': 99.77351185685663,
 'product_16': 99.888900080862,
 'product_17': 99.72459550173397,
 'product_18': 99.73321683953662,
 'product_21': 99.70525818289653,
 'product_22': 99.67087094889841,
 'product_24': 99.72473076349623,
 'product_25': 99.77475794485306,
 'product_26': 99.77691739503084,
 'product_28': 99.74746513923736,
 'product_30': 99.78092567999605,
 'product_34': 99.53870252554464,
 'product_44': 99.64219376638664,
 'product_46': 99.724110844987,
 'product_49': 99.7016359320751,
 'product_50': 99.69972142462001,
 'product_54': 99.38945060120388,
 'product_55': 99.75881945581868,
 'product_59': 99.78688336722658,
 'product_60': 99.83975784996

In [28]:
std_dev_dict

{'product_1': 0.0007971500863566995,
 'product_2': 0.0006295572669572843,
 'product_3': 0.0008930028140111209,
 'product_4': 0.0007936715929608968,
 'product_5': 0.0010349480758214413,
 'product_6': 0.0009139173586310459,
 'product_9': 0.000607510989256313,
 'product_12': 0.0008159442480420033,
 'product_13': 0.0005453434960305636,
 'product_14': 0.0005782418582277027,
 'product_15': 0.0009081111023812931,
 'product_16': 0.0006878393334082845,
 'product_17': 0.0007016528404350532,
 'product_18': 0.0008913276415014895,
 'product_21': 0.0007712075490503697,
 'product_22': 0.0009222821632734135,
 'product_24': 0.0009547655808455198,
 'product_25': 0.0007460082598058692,
 'product_26': 0.0009297188534779628,
 'product_28': 0.0010380674854430282,
 'product_30': 0.0007610506354431031,
 'product_34': 0.0033250855595784922,
 'product_44': 0.0011470299748743064,
 'product_46': 0.0009197915139389147,
 'product_49': 0.0009805159987854263,
 'product_50': 0.0012731317855422784,
 'product_54': 0.002